# Introdução

Este *notebook* tem como objetivo realizar o primeiro acesso aos dados, identificar o ponto de separação entre os dados que serão usados para treinamento e teste e, por fim, materializar a separação com a criação dos conjuntos de dados.

# Bibliotecas e Funções

In [1]:
# Usado para ler arquivos, carregar código personalizado e ter acesso a outros recursos de sistema
import os
import sys

# Manipulação e análise dos dados
import pandas as pd

# Recursos para visualização dos dados
import matplotlib.pyplot as plt
from IPython.core.display import HTML, display

# Carregar código personalizado disponível em ../src
sys.path.append(os.path.abspath(os.path.pardir))
from src import settings
from src.utils.notebooks import display_side_by_side
from src.utils.experiments import set_dataset_split

# Configurações para a exibição de conteúdo do Pandas e das bibliotecas gráficas
%matplotlib inline 
pd.set_option('display.max_rows', None)
pd.set_option("display.max_columns", None)
pd.set_option('max_colwidth', 150)

# Organização da Estrutura e Download do Conjunto de Dados

Para iniciar o projeto, é preciso criar a estrutura de diretórios de dados, além de baixar e persistir o conjunto de dados no local de referência.

In [2]:
# Criar estrutura de diretórios, caso não exista
if not os.path.exists(settings.DATA_PATH):
    os.makedirs(settings.DATA_PATH)

for directory in ['raw', 'interim', 'processed']:
    path = os.path.join(settings.DATA_PATH, directory)
    if not os.path.exists(path):
        os.makedirs(path)

raw_dataset_path = os.path.join(settings.DATA_PATH, 'raw', 'elo7_recruitment_dataset.csv')

# Baixar e persistir o conjunto de dados, caso ele ainda não esteja em disco
if not os.path.exists(raw_dataset_path):
    raw_dataset = pd.read_csv(settings.RAW_DATASET_PATH)
    assert len(raw_dataset) == 38_507, f'Os dados acessados não têm o número esperado de registros: {len(raw_dataset)} != 38.507).'
    raw_dataset.to_csv(raw_dataset_path, index=False)
    del raw_dataset

# Análise Preliminar

Este é o primeiro ponto de contato com os dados, usado para identificar a quantidade de registros, as colunas disponíveis e os tipos de dados envolvidos. Um ponto importante neste momento é ter o **entendimento inicial do que existe** e **determinar um ponto de corte nos dados para evitar o vazamento de informações** do que será o **conjunto de teste**. 

Separar o conjunto de teste antes mesmo de fazer a análise exploratória completa dos dados é o melhor jeito de simular o que acontecerá com o(s) modelo(s) quando ele for utilizado em produção com dados novos e, com isso, evitar uma estimativa irreal dos resultados.


## Carregamento dos dados

Leitura do arquivo completo disponível.

In [3]:
frame = pd.read_csv(raw_dataset_path, infer_datetime_format=True)

## Visualização dos primeiros registros disponíveis

Nesta primeira visão, é possível entender que o conjunto de dados é contextualizado na sua posição com relação à busca. Mantendo o foco na separação do conjunto de teste, esperaria ter a data da busca para conseguir separar os dados no tempo. A única informação de tempo disponível se refere à criação do produto.

In [4]:
print(f'Total de registros: {len(frame)}')
frame.head(15)

Total de registros: 38507


,product_id,seller_id,query,search_page,position,title,concatenated_tags,creation_date,price,weight,express_delivery,minimum_quantity,view_counts,order_counts,category
0,11394449,8324141,espirito santo,2,6,Mandala Espírito Santo,mandala mdf,2015-11-14 19:42:12,171.890000,1200.0,1,4,244,NaN,Decoração
1,15534262,6939286,cartao de visita,2,0,Cartão de Visita,cartao visita panfletos tag adesivos copos long drink canecas,2018-04-04 20:55:07,77.670000,8.0,1,5,124,NaN,Papel e Cia
2,16153119,9835835,expositor de esmaltes,1,38,Organizador expositor p/ 70 esmaltes,expositor,2018-10-13 20:57:07,73.920006,2709.0,1,1,59,NaN,Outros
3,15877252,8071206,medidas lencol para berco americano,1,6,Jogo de Lençol Berço Estampado,t jogo lencol menino lencol berco,2017-02-27 13:26:03,118.770004,0.0,1,1,180,1.0,Bebê
4,15917108,7200773,adesivo box banheiro,3,38,ADESIVO BOX DE BANHEIRO,adesivo box banheiro,2017-05-09 13:18:38,191.810000,507.0,1,6,34,NaN,Decoração
5,4336889,3436479,dia dos pais,1,37,Álbum de figurinhas dia dos pais,albuns figurinhas pai lucas album fotos,2018-07-11 10:41:33,49.970000,208.0,1,1,1093,NaN,Lembrancinhas
6,7544556,7118324,arranjo de flores para mesa,1,9,Arranjo de Flores - Orquidias,mini arranjos,2016-04-22 13:34:16,23.670000,207.0,1,5,276,NaN,Decoração
7,10869150,5203458,lembrancinha maternidade,5,18,Kit Aromarizador + sacola / Lembrancinha Maternidade,bb lembrancinhas maternidade baby lembranca maternidade bebe conforto lembrancinha maternidade,2017-10-05 00:26:02,12.710000,55.0,0,33,1178,109.0,Lembrancinhas
8,13193769,2933585,chaveiro dia dos pais,1,35,chaveiro dia dos pais,dia pais,2018-07-04 12:47:49,11.420000,6.0,1,23,72,NaN,Lembrancinhas
9,13424151,8530613,manta personalizada,1,20,Manta para bebê personalizada de Nuvem com nome,nascimento manta baby cha bebe vestido bebe,2018-04-03 16:10:51,107.100000,9.0,1,1,639,26.0,Bebê


Para confirmar o significado da data com relação ao produto, além de ter a noção de como outras colunas se comportam, é possível fazer a verificação da repetição dos produtos (tabela à esquerda/do topo) e, para o produto mais repetido, verificar as informações disponíveis (tabela à direita/de baixo).

In [5]:
# Criar data frame com o total de ocorrência de cada produto
agg_frame = (
    frame
    [['product_id']]
    .assign(registros=1)
    .groupby('product_id')
    .sum()
    .reset_index()
    .sort_values(by='registros', ascending=False)
)

# Guardar o ID do produto com maior número de ocorrências
most_frequent_product = agg_frame.iloc[0]['product_id']

# Criar dataframe com os registros do produto com maior ocorrência
summary_frame = (
    frame
    .loc[lambda f: f['product_id'] == most_frequent_product]
    .describe(include='all')
    .T
)

# Exibição dos resultados
display_side_by_side([agg_frame.head(20), summary_frame],
                     ['Produtos mais repetidos', 'Resumo Histórico do produto mais repetido'],
                     padding=50)

del agg_frame, summary_frame  # remoção dos dataframes intermediários

,product_id,registros
13477,7557702,14
19791,11130723,12
7585,4225282,12
16150,9059093,11
22683,12716324,11
25299,14155651,11
407,240412,11
5354,2952037,11
17326,9715550,11
9606,5395219,10


Conforme pode ser observado, o produto com maior ocorrência nos dados tem 14 registros. A repetição da data de criação, que é única para todos, o que reforça o entendimento de que se refere à criação do produto e não da busca ou de outro elemento. Os elementos mais próximos de tempo com relação à busca são as colunas de visualizações (*view_counts*) e de pedidos (*order_counts*). Porém, o total de pedidos não tem crescimento monónoto com relação ao total de visualizações. Isto ocorre, de acordo [com a documentação dos dados](https://github.com/SamaraAlves/teste_HT#dataset), porque **os contadores consideram a janela dos últimos 3 meses**.

Um **ponto de alerta**, com relação ao **vazamento de dados**, é que existem duas buscas para o mesmo produto e com total de visualizações diferentes ('presente dia dos pais'). O fato de a busca estar relacionada a uma data do ano reforça a preocupação com aspectos de sazonalidade e diferenças de resultados ao longo do tempo.

O fato de um mesmo produto ter variações de preço e peso é algo curioso. Como o produto tem o mesmo vendedor, é possível entender que houve uma mudança na composição do produto ao longo do tempo e isso gerou a mudança -- e isto não está direcionamente relacionado à quantidade mínima do pedido, já que a mudança do peso não é proporcional à quantidade. A mudança no preço, que também apresentou flutuações, já deve ser esperada em função de promoções e reajustes de preço.

In [6]:
# Filtrar dados para considerar apenas o produto mais frequente e 
# ordenar colunas por visualizações e pedidos
(
    frame
    .loc[lambda f: f['product_id'] == most_frequent_product]
    .sort_values(by=['view_counts', 'order_counts'])
)

del most_frequent_product

## Separação dos dados de Treinamento e Teste

Considerando a visão inicial dos dados e o **entendimento de que a data de criação é a melhor referência para separar os dados entre treinamento e teste**, é preciso analisar como definir o ponto de corte.

A seguir, faz-se a análise do total de produtos disponíveis nos dados caso um determinado período (a combinação de ano-mês) seja utilizado.

In [7]:
def compute_cumulative_products(frame: pd.DataFrame) -> pd.DataFrame:

    agg_frame = (
        frame        
        .assign(period=lambda f: f['creation_date'].apply(lambda x: str(x)[:7]))
        .sort_values(by='period')
        [['period']]
        .assign(produtos=1)
        .groupby('period')
        .sum()
        .reset_index()
        .assign(produtos_totais=lambda f: f['produtos'].cumsum())
        .assign(perc=lambda f: (100 * f['produtos_totais'] / f['produtos'].sum()).apply(lambda x: f'{x:.2f}%'))
    )
    
    return agg_frame

records_frame = compute_cumulative_products(frame).tail(25)
products_frame = compute_cumulative_products(frame.drop_duplicates('product_id')).tail(25)
difference_frame = pd.DataFrame({'Diferença': records_frame['produtos_totais'] - products_frame['produtos_totais']})

display_side_by_side([records_frame, products_frame, difference_frame],
                     ['Registros (A)', 'Produtos Únicos (B)', 'A - B (p/ produtos_totais)'], padding=50)

del records_frame, products_frame, difference_frame

,period,produtos,produtos_totais,perc
105,2017-11,1067,19615,50.94%
106,2017-12,874,20489,53.21%
107,2018-01,1169,21658,56.24%
108,2018-02,1251,22909,59.49%
109,2018-03,1303,24212,62.88%
110,2018-04,1290,25502,66.23%
111,2018-05,1642,27144,70.49%
112,2018-06,1876,29020,75.36%
113,2018-07,2088,31108,80.79%
114,2018-08,1638,32746,85.04%


Na tabela à esquerda (*Registros (A)*), é possível observar a totalização de registros, considerando inclusive as repetições de um mesmo produto. Ao centro (*Produtos Únicos (B)*), cada produto é contabilizado de forma única, para que seja possível determinar se a diferença na quantidade de dados será grande para cada ponto de corte. À direita (*A- B (p/ produtos_totais)*) está a diferença absoluta entre os registros acumulados até o período de corte.

Para que se tenha uma quantidade razoável de dados para validar os modelos e experimentos, o ponto de corte definido é para o período de **2018-08**, o que reserva em torno de **15%** dos registros para teste.

Cabe ressaltar que para este conjunto de dados a escolha do ponto de corte representaria uma diferença de 17 meses entre os dados de treino mais recentes e os dados de teste mais recentes. Para um modelo em produção, provavelmente o modelo seria re-treinado algumas vezes nesse período, para acompanhar a mudança do comportamento dos vendedores e seus produtos. Apesar disso, esse período longo de dados de teste reduz (ainda que não evite totalmente) o viés de se ter apenas produtos muito recentes nos dados de teste -- algo que também é amenizado por se ter contadores com uma janela máxima de 3 meses.

Logo abaixo os dados são separados com esse ponto de referência estabelecido.

In [8]:
def compute_split_distribution(frame: pd.DataFrame) -> pd.DataFrame:    
    agg_frame = (
        frame
        [['group']]
        .assign(registros=1)
        .groupby('group')
        .sum()
        .reset_index()
        .assign(perc=lambda f: ( 100 * f['registros'] / f['registros'].sum()).apply(lambda x: f'{x:.2f}%'))
    )

    return agg_frame


cut_off_period = '2018-08'
split_frame = set_dataset_split(frame, cut_off_period)
agg_records = compute_split_distribution(split_frame)
agg_products = compute_split_distribution(split_frame.drop_duplicates('product_id'))

display(HTML(f'<h3>Separação de conjuntos de dados para o período de corte: {cut_off_period}</h3>'))

display_side_by_side([agg_records, agg_products],
                     ['Registros', "Produtos Únicos"],
                     padding=50
                    )
del agg_records, agg_products

,group,registros,perc
0,test,5761,14.96%
1,training,32746,85.04%
,group,registros,perc
0,test,4645,15.59%
1,training,25156,84.41%


Para garantir que a divisão seja válida e não possua alguma limitação com relação às classes disponíveis, duas validações:
 - Não há períodos em comum entre os dois conjuntos;
 - As categorias do conjunto de teste também existem no conjunto de treinamento.

## Persistência dos Dados de Treinamento e de Teste

Logo abaixo, os dois conjuntos de dados são verificados com relação à separação de períodos e produtos.

In [9]:
training_frame = split_frame.loc[lambda f: f['group'] != 'test'].drop(columns=['group'])
test_frame = split_frame.loc[lambda f: f['group'] == 'test'].drop(columns=['group'])
del split_frame

assert len(set(test_frame['category'].tolist()) - set(training_frame['category'].tolist()) ) == 0, \
'Há categorias do grupo de teste que não estão no grupo de treinamento.'

assert len(set(test_frame['period'].tolist()) & set(training_frame['period'].tolist()) ) == 0, \
'Há períodos compartilhados entre os dados de treinamento e de teste.'

Considernado que as premissas básicas não foram violadas, os conjuntos de dados serão persistidos para uso nas outras etapas.

In [10]:
training_frame.to_csv(os.path.join(settings.DATA_PATH, 'interim', 'training.csv'), index=False)
test_frame.to_csv(os.path.join(settings.DATA_PATH, 'interim', 'test.csv'), index=False)